In [96]:
## import packages
from util import createGrps
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms import Predictor
from lenskit.algorithms.user_knn import UserUser
from lenskit import batch
import sys

In [97]:
## Get the training data and train the user-user collaborative filterring
# read the data (about applications)
# TODO: replace by Tine's filter  data
data = pd.read_csv('../dataset/user_with_negative_ratings_full.csv', delimiter=',')

# load user data
user_data = pd.read_csv('../dataset/users.tsv', sep='\t')

In [98]:
def cap(x):
    if x > 1.0:
        return 1.0
    elif x < 0.0:
        return 0.0
    return x 

In [99]:
## First get the groups
# parameters for group generation
size = 10
grpNum = 20
seed = 12345

# generate synthetic groups of users and display them
groups2 = createGrps(user_data, size, grpNum)
display(groups2)
print()

,Users
0,"[872491.0, 947984.0, 534411.0, 351315.0, 20558..."
1,"[1186664.0, 93353.0, 1237855.0, 31806.0, 53279..."
2,"[589862.0, 755811.0, 1085196.0, 376755.0, 1097..."
3,"[24742.0, 180921.0, 101397.0, 100719.0, 121090..."
4,"[512772.0, 268729.0, 1358530.0, 1211674.0, 117..."
...,...
15,"[1368294.0, 490913.0, 1239385.0, 1288222.0, 14..."
16,"[861160.0, 1192692.0, 884578.0, 281648.0, 1253..."
17,"[610206.0, 19333.0, 457900.0, 1215441.0, 48373..."
18,"[502559.0, 725561.0, 239339.0, 893181.0, 10829..."


In [100]:
# construct dataframe in format (user, item, rating) via column addition
df_ui = data.rename(columns={"UserID": "user", "JobID": "item", "Rating":"rating"})
# check data being read properly
display(df_ui.head(10)) 

# train UserUser collaborative filterring
user_user = UserUser(10, min_nbrs=3)  # Minimum (3) and maximum (10) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

,user,item,rating
0,47,169528,1
1,47,284009,1
2,47,2121,1
3,47,848187,1
4,47,733748,1
5,47,576958,1
6,47,262470,1
7,47,602298,1
8,47,982331,0
9,47,979937,0


In [101]:
# pd.set_option('display.max_rows', None)
all_ratings = []

## Create a User-Item matrix of scores so we can apply one of the aggregation strategies
# iterate through groups generated
for i, row in groups2.iterrows():
    # get the array of users from the row
    synGrp = row.iloc[0]

    # get recommendations for all group members
    ratings_grp = batch.recommend(recsys, synGrp, n=None,  n_jobs=1)
    
    # cap the values (x > 1 then 1; x < 0 then 0)
    # TODO: if you are confident that you can explain the clamping -> go ahead and use it
    # ratings_grp["score"] = ratings_grp["score"].map(cap)
    # print(ratings_grp)
    # TODO: sys.exit only for demonstration to not run through all 20 groups of users
    all_ratings.append((synGrp, ratings_grp))


user 418304 has no ratings and none provided
user 6981 has no ratings and none provided
user 379846 has no ratings and none provided
user 589862 has no ratings and none provided
user 100719 has no ratings and none provided
user 1358530 has no ratings and none provided
user 1211674 has no ratings and none provided
user 624450 has no ratings and none provided
user 1010047 has no ratings and none provided
user 1436734 has no ratings and none provided
user 258138 has no ratings and none provided
user 167779 has no ratings and none provided
user 957318 has no ratings and none provided
user 1205916 has no ratings and none provided
user 743958 has no ratings and none provided
user 299674 has no ratings and none provided
user 359861 has no ratings and none provided
user 645552 has no ratings and none provided
user 917070 has no ratings and none provided
user 913507 has no ratings and none provided
user 1284755 has no ratings and none provided
user 608671 has no ratings and none provided
user 1

In [102]:
num_pref = 5
group_preference = []
index = 0

for label, ratings_grp in all_ratings:
    # Print the label to differentiate the DataFrames
    print("Label:", label)
    
    # Access and work with the ratings_grp DataFrame
    display(ratings_grp.head(10))  # Example: Display the first few rows of the DataFrame
    
    # Perform any other operations or analysis on the DataFrame as needed
    unique_items = ratings_grp['item'].unique().tolist()

    for item in unique_items:
        job_scores = ratings_grp[ratings_grp['item'] == item]
        if len(job_scores) > 1:
            print("Number of job_scores:", len(job_scores))
        
    index += 1
    sys.exit()

Label: [ 872491.  947984.  534411.  351315.  205589.  952791. 1024927.  439017.
  793281.  498700.]


,item,score,user,rank
0,1115111,-0.122551,872491.0,1
1,1105728,-0.135268,872491.0,2
2,1111238,-0.195580,872491.0,3
3,1115522,-0.207330,872491.0,4
4,765794,0.913294,534411.0,1
5,259416,0.844203,534411.0,2
6,854256,0.830390,534411.0,3
7,287853,0.789351,534411.0,4
8,806520,0.769054,534411.0,5
9,879068,0.767842,534411.0,6


Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2
Number of job_scores: 2


SystemExit: 

c:\Users\domin\anaconda3\envs\rcsys\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
